In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras_tuner import Hyperband  # Install keras-tuner if not installed: pip install keras-tuner

# Define the model-building function for hyperparameter tuning
def build_model(hp):
    model = models.Sequential()
    
    # Convolutional layers with tunable hyperparameters
    model.add(layers.Conv2D(
        filters=hp.Int('conv1_units', min_value=32, max_value=128, step=16),  # Tunable number of filters
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(
        filters=hp.Int('conv2_units', min_value=32, max_value=128, step=16),
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=128, step=16),  # Tunable dense units
        activation='relu'
    ))
    model.add(layers.Dropout(
        rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)  # Tunable dropout rate
    ))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer with 10 classes

    # Optimizer with tunable learning rate
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])  # Tunable learning rate
    )
    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # Use from_logits=False for softmax
        metrics=['accuracy']
    )
    return model

# Load the CIFAR-10 dataset
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize the data
x_train, x_val = x_train / 255.0, x_val / 255.0

# Define the tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',  # Metric to optimize
    max_epochs=10,  # Maximum number of epochs per trial
    factor=3,  # Reduction factor for the number of configurations
    directory='hyperband_dir',  # Directory to save results
    project_name='cifar10_tuning'  # Project name
)

# Run the search for the best hyperparameters
tuner.search(
    x_train, y_train,
    epochs=10,
    validation_data=(x_val, y_val)
)

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Display the optimal hyperparameters
print(f"""
The optimal number of filters in the first Conv2D layer is {best_hps.get('conv1_units')}.
The optimal number of filters in the second Conv2D layer is {best_hps.get('conv2_units')}.
The optimal number of units in the dense layer is {best_hps.get('dense_units')}.
The optimal dropout rate is {best_hps.get('dropout_rate')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the best model
history = model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_val, y_val)
)

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f"Validation accuracy: {val_accuracy:.4f}")


Trial 7 Complete [00h 02m 54s]
val_accuracy: 0.5139999985694885

Best val_accuracy So Far: 0.6581000089645386
Total elapsed time: 00h 24m 39s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
112               |48                |conv1_units
80                |128               |conv2_units
112               |128               |dense_units
0.3               |0.1               |dropout_rate
0.01              |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 132s 82ms/step - accuracy: 0.1025 - loss: 2.3632 - val_accuracy: 0.1000 - val_loss: 2.3031
Epoch 2/2
 395/1563 ━━━━━━━━━━━━━━━━━━━━ 1:44 89ms/step - accuracy: 0.0992 - loss: 2.3040